## This code contain a quick method for creating a structured dataset from atmospheric data.

In [ ]:
#Generate Atmosphere datasets from era5

import cdsapi
c = cdsapi.Client()
namefile = 'u'
variabel = 'u_component_of_wind'
level = ['925','850','700']
yi = 2000
yt = 2022
dataset = 'reanalysis-era5-pressure-levels'
for tahun in range(yi,yt+1):
    c.retrieve(
        dataset,
        {
            'product_type': 'reanalysis',
            'variable': variabel,
            'pressure_level': level,
            'year': [str(tahun)],
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'day': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
                '13', '14', '15',
                '16', '17', '18',
                '19', '20', '21',
                '22', '23', '24',
                '25', '26', '27',
                '28', '29', '30',
                '31',
            ],
            'time': [
                '00:00', '01:00', '02:00',
                '03:00', '04:00', '05:00',
                '06:00', '07:00', '08:00',
                '09:00', '10:00', '11:00',
                '12:00', '13:00', '14:00',
                '15:00', '16:00', '17:00',
                '18:00', '19:00', '20:00',
                '21:00', '22:00', '23:00',
            ],
            'area': [
                -5, 106, -7, 108,
            ],
            'grid': ['1', '1'],
            'format': 'netcdf',
        },
        f'./ncfile/{namefile}{tahun}.nc')

In [ ]:
import xarray as xr
import pandas as pd
import os

datadir = './ncfile/'
netcdf_file = []

# Check Folder
if os.path.exists(datadir):
    #Loop list file
    for filename in os.listdir(datadir):
        netcdf_file.append(filename)

    print("List of files in the folder:")
    print(file_list)
else:
    print(f"Folder '{datadir}' Not Found.")

In [18]:

# List of netCDF files
netcdf_files = ['airdensity.nc', 'cbh.nc', 'dew2m.nc', 'evaporation.nc', 'firstswell.nc', 'geo500.nc', 'geo700.nc',
             'geo850.nc', 'geo925.nc', 'hcc.nc', 'lcc.nc', 'mcc.nc', 'mslp.nc', 'olr.nc', 'rh500.nc',
             'rh700.nc', 'rh850.nc', 'rh925.nc', 'runoff.nc', 'secondswell.nc', 'sh500.nc', 'sh700.nc', 'sh850.nc',
             'sh925.nc', 'skt.nc', 'sp.nc', 'sst.nc', 'surfacesolar.nc', 'surfacethermal.nc', 't2m.nc', 't500.nc',
             't700.nc', 't850.nc', 't925.nc', 'tcc.nc', 'tclw.nc', 'tcwv.nc', 'topsolar.nc', 'u10.nc', 'u500.nc',
             'u700.nc', 'u850.nc', 'u925.nc', 'v10.nc', 'v500.nc', 'v700.nc', 'v850.nc', 'v925.nc',
             'verticalvelocity500.nc', 'verticalvelocity700.nc', 'verticalvelocity850.nc', 'verticalvelocity925.nc',
             'viclwf.nc', 'vimd.nc', 'vit.nc', 'vithermal.nc', 'vorticity500.nc', 'vorticity700.nc', 'vorticity850.nc',
             'vorticity925.nc', 'waveperiod.nc', 'windwavedirection.nc', 'windwaveperiode.nc']

# Specify the area of interest (replace with some coordinates)
latitude_range = slice(25, 30)
longitude_range = slice(125, 135)

# Function to calculate average over the specified area
def calculate_average(file_path):
    ds = xr.open_dataset(datadir+file_path)
    avg_data = pd.Series((ds.sel(lon=longitude_range,lat=latitude_range).fillna(0).mean(("lon", "lat"), skipna=True)).to_array().values.flatten())
    ds.close()
    return avg_data

# Create a DataFrame to store the results
df = pd.DataFrame()

# Loop through each netCDF file and calculate the average
for file_path in netcdf_files:
    variable_name = file_path.split('.')[0]  # Extract variable name from the file path
    avg_data = calculate_average(file_path)
    df[variable_name+'_125-135.25-30'] = avg_data

# Add the time index to the DataFrame
df['time'] = xr.open_dataset(datadir+file_path).time
df.set_index('time', inplace=True)

# Display the resulting DataFrame
print(df)


            airdensity_125-135.25-30  cbh_125-135.25-30  dew2m_125-135.25-30  \
time                                                                           
2018-01-01                  1.226213        1376.388306           281.285095   
2018-01-02                  1.222755        1395.283569           281.591583   
2018-01-03                  1.219708        1368.042847           282.892761   
2018-01-04                  1.209923        1579.050049           284.719910   
2018-01-05                  1.208615         906.702393           285.968231   
...                              ...                ...                  ...   
2022-12-27                  1.221846        1158.566895           282.780334   
2022-12-28                  1.220156         977.267639           284.863586   
2022-12-29                  1.221603        1090.648560           285.594818   
2022-12-30                  1.221764         950.300903           285.839447   
2022-12-31                  1.225944    

In [44]:
precip=pd.read_csv('rainfall.csv')
precip['time']=xr.open_dataset(datadir+file_path).time
precip.set_index('time', inplace=True)

In [45]:
""""Because I am using data from multiple locations, I have to define df1, df2, df3, and so on, and then merge them.
Pandas cannot generate hundreds of series simultaneously."""
merge=pd.concat([precip['precip'],df,df1,df2,df3,df4,df5,df6,df7,df8,df9],axis=1)
merge

,precip,airdensity_local,cbh_local,dew2m_local,evaporation_local,firstswell_local,geo500_local,geo700_local,geo850_local,geo925_local,...,vimd_125-135.25-30,vit_125-135.25-30,vithermal_125-135.25-30,vorticity500_125-135.25-30,vorticity700_125-135.25-30,vorticity850_125-135.25-30,vorticity925_125-135.25-30,waveperiod_125-135.25-30,windwavedirection_125-135.25-30,windwaveperiode_125-135.25-30
time,,,,,,,,,,,,,,,,,,,,,
2018-01-01,0.548293,0.766984,885.471924,296.373657,-0.000195,0.330177,57284.187500,30596.742188,14507.958008,7329.496582,...,0.450078,2643711.00,2.659043e+09,-0.000003,-0.000010,-0.000014,-0.000016,6.565504,138.899109,4.573166
2018-01-02,11.442953,0.767079,791.576904,296.398254,-0.000155,0.305726,57374.113281,30632.871094,14530.801758,7353.888184,...,-0.015954,2652803.25,2.669318e+09,-0.000019,-0.000013,-0.000026,-0.000019,7.383528,110.016106,3.211693
2018-01-03,16.924893,0.767366,736.118408,296.733429,-0.000146,0.210228,57444.675781,30687.673828,14574.935547,7392.477539,...,0.154344,2654569.50,2.672966e+09,-0.000035,-0.000028,-0.000024,-0.000015,6.623868,145.354050,3.993666
2018-01-04,10.931562,0.768230,1094.648071,296.646149,-0.000149,0.231009,57427.957031,30741.076172,14622.370117,7441.261230,...,-0.426703,2644710.25,2.664634e+09,-0.000025,-0.000030,-0.000017,-0.000006,6.921987,107.194000,3.142183
2018-01-05,4.713392,0.770521,1148.417480,296.304626,-0.000161,0.259345,57348.503906,30710.460938,14644.161133,7481.770996,...,0.862925,2635334.25,2.654212e+09,-0.000010,-0.000009,0.000011,0.000010,6.560161,238.952606,5.108466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,12.484982,0.773419,1106.524780,295.776184,-0.000196,0.419686,57539.351562,30901.392578,14855.633789,7711.625977,...,0.106120,2655763.50,2.673041e+09,-0.000011,-0.000004,-0.000005,-0.000003,6.683178,115.973267,3.187070
2022-12-28,38.176224,0.774339,916.623779,296.079987,-0.000215,0.350781,57519.023438,30883.232422,14828.390625,7683.097168,...,0.126929,2660355.50,2.678408e+09,-0.000028,-0.000025,-0.000015,-0.000009,6.528627,132.862320,3.825282
2022-12-29,37.825348,0.773788,403.953156,296.296753,-0.000225,0.400518,57501.398438,30861.902344,14810.533203,7663.331543,...,-0.171881,2662063.00,2.681606e+09,-0.000028,-0.000029,-0.000021,-0.000015,6.726167,69.463745,4.584798


In [47]:
#Delete column with null data & Save
merge = merge.loc[:, (merge != 0).any(axis=0)]
merge.to_csv('completedata.csv')
